# 文本生成

In [ ]:
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path
current_foldr=!pwd
current_foldr=current_foldr[0]
current_foldr

## 確認CUDA版本, 以及否能使用GPU
若無gpu 請點選右側->已連線->變更執行階段類型->T4 Gpu

In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

## 安裝套件
安裝完成後建議, 點選上方選單, 直接階段->重新啟動工作階段, 確保 library重置

In [ ]:
!pip install chromadb cohere gdown kaleido langchain openai pyngrok pypdf python-dotenv sentence-transformers tiktoken -q
!pip install accelerate bitsandbytes hf_transfer huggingface_hub optimum transformers -q 
!pip install -r requirements.txt -q

## 模型下載

## 程式碼啟動 WEB

In [ ]:
%%bash
cat << EOF >  src/web_demo_test.py
from llmtuner import create_web_demo


def main():
    demo = create_web_demo()
    demo.queue()
    demo.launch(server_port=9000, server_name="$(hostname -s)", share=False, inbrowser=True)


if __name__ == "__main__":
    main()
EOF

In [ ]:
%%bash
node_hostname=$(hostname -s)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
#noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=9000

# SSH FORWARDING
ssh_cmd="ssh -L ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"
echo "SSH:"
echo ${ssh_cmd}
echo ""
echo "URL: http://localhost:${noed_port_genai}"
echo ""
 
# WEB
CUDA_VISIBLE_DEVICES=0 HF_TOKEN='hf_' \
python src/web_demo_test.py \
    --model_name_or_path /work/g00cjz00/models/mistralai/Mixtral-8x7B-Instruct-v0.1 \
    --checkpoint_dir /work/g00cjz00/models/wangrongsheng/Aurora \
    --finetuning_type lora \
    --quantization_bit 4 \
    --template mistral

## 程式碼啟動 API

In [ ]:
%%bash
cat << EOF >  src/api_demo_test.py
import uvicorn
from llmtuner import ChatModel, create_app

def main():
    chat_model = ChatModel()
    app = create_app(chat_model)
    print("Visit http://localhost:9000/docs for API document.")
    uvicorn.run(app, host="$(hostname -s)", port=9000, workers=1)

if __name__ == "__main__":
    main()
EOF

In [ ]:
%%bash
node_hostname=$(hostname -s)
node_ip=$(cat /etc/hosts |grep "$(hostname -a)" | awk '{print $1}')
# PORT
#noed_port_genai=$(python -c "import socket; s = socket.socket(socket.AF_INET, socket.SOCK_STREAM); s.bind(('', 0)); addr = s.getsockname(); s.close(); print(addr[1])")
noed_port_genai=9000

# SSH FORWARDING
ssh_cmd="ssh -L ${noed_port_genai}:${node_hostname}:${noed_port_genai} $(whoami)@t3-c4.nchc.org.tw"
echo "SSH:"
echo ${ssh_cmd}
echo ""
echo "URL: http://localhost:${noed_port_genai}"
echo ""
 
# API
CUDA_VISIBLE_DEVICES=0 HF_TOKEN='hf_' \
python src/api_demo_test.py \
    --model_name_or_path /work/g00cjz00/models/mistralai/Mixtral-8x7B-Instruct-v0.1 \
    --checkpoint_dir /work/g00cjz00/models/wangrongsheng/Aurora \
    --finetuning_type lora \
    --quantization_bit 4 \
    --template mistral

In [ ]:
##OEPNAPI
#指定ChatGLM2-6B的API endpoint url，用langchain的ChatOpanAI类初始化一个ChatGLM的chat模型
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
        openai_api_base="http://gpn3002:9000/v1",
        openai_api_key="EMPTY",
        streaming=True,
    )

#使用会话实体内存，利用ChatGLM在会话过程中分析提到的实体(Entity)
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
entity_memory = ConversationEntityMemory(llm=llm, k=5 )

#生成会话链
from langchain.chains import ConversationChain
Conversation = ConversationChain(
            llm=llm, 
            prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
            memory=entity_memory,
            verbose=True,
        ) 
#开始测试
#Conversation.run("寫一篇關於黃昏的描述, 要求細緻生動, 景色美好")
Conversation.run("小名手上有100顆蘋果, 送給小君10個, 又送給小林9個, 請問小名手上剩下幾個蘋果")

## Training

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python src/train_bash.py \
    --stage sft \
    --do_train \
    --model_name_or_path /work/g00cjz00/models/mistralai/Mixtral-8x7B-Instruct-v0.1 \
    --dataset alpaca_gpt4_zh \
    --template mistral \
    --finetuning_type lora \
    --quantization_bit 4 \
    --lora_target q_proj,v_proj \
    --output_dir path_to_sft_checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 100 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16


## Evaluation

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python src/evaluate.py \
    --model_name_or_path /work/g00cjz00/models/mistralai/Mixtral-8x7B-Instruct-v0.1 \
    --checkpoint_dir /work/g00cjz00/models/wangrongsheng/Aurora/checkpoint-5000 \
    --template mistral \
    --finetuning_type lora \
    --quantization_bit 4 \
    --task cmmlu \
    --split test \
    --lang en \
    --n_shot 5 \
    --batch_size 8